Importing the libraries

In [ ]:
import os
import tensorflow
from keras import layers
from keras.models import Model
from keras import optimizers
from keras import metrics
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3


Loading the model with an input of (197,197,3), where 197 is the width/height and 3 is the number of color channels(RGB)


In [ ]:
pre_trained_model = InceptionV3(input_shape = (197,197, 3),
                                include_top = False,
                                weights = 'imagenet' )

Layer definition 

In [ ]:
for layer in pre_trained_model.layers:
  layer.trainable = False
  

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_layer.trainable = True
last_output = last_layer.output


x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation = 'relu')(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation = 'relu')(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(2, activation = 'softmax')(x)

model = Model(pre_trained_model.input, x)


Defining training parameter

In [ ]:
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(optimizer = 'Adam',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

Directory of the training/Validation images used by ImageDataGenerator

In [ ]:
train_dir = '/content/sample_data/AfterShoot/first'
validation_dir = '/content/sample_data/AfterShoot/second'
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                  rotation_range = 40,
                                  horizontal_flip = True,
                                  vertical_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                   batch_size = 100,
                                                   class_mode = 'categorical',
                                                   target_size = (299, 299))
# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                   batch_size = 20,
                                                   class_mode = 'categorical',
                                                   target_size = (299, 299))



Fitting the data into the model to start the training process

In [ ]:
history = model.fit_generator(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 100, validation_steps = 20, verbose = 1)

#Saving the model
Use hd5 extension to save all the model checkpoints, use no extension if only the model is required to do the final testing/implementation

In [ ]:
model.save(<path>)

Plotting the accuracy/validation metrics over epoch

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
fig1 = plt.gcf()
plt.show()
plt.draw()
